In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0),(0,0,0)),
                INDY_IP)]
              , connection_list=[False])

connection_list
[False]


## init scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [4]:
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


In [13]:
gscene.show_pose(np.array([np.pi/3        ,  0.        , -np.pi/2,  -np.pi/4        , -np.pi/3,
        0.        ]))

## add environment

In [5]:
from pkg.geometry.geotype import GEOTYPE

In [6]:
floor = gscene.create_safe(gtype=GEOTYPE.BOX, name="floor", link_name="base_link", 
                 dims=(0.5,0.5,0.3), center=(0.5,0,0.15), rpy=(0,0,0), color=(0.8,0.8,0.8,1), display=True, collision=True, fixed=True)

Please create a subscriber to the marker


In [7]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=False, collision=True)

## add movable

In [8]:
box = gscene.create_safe(gtype=GEOTYPE.BOX, name="box", link_name="base_link", 
                         dims=(0.05,0.1,0.2), center=(0.5,0,0.4), rpy=(0,0,np.pi/2), color=(0.8,0.2,0.2,1), display=True, collision=True, fixed=True)

## init planning scene

In [9]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, rname="indy0", point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import BoxObject
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

In [14]:
HANDLE_THICKNESS = 1e-6
GRIP_DEPTH = 5e-2
CLEARANCE = 1e-3
HANDLE_COLOR = (1,0,0,0)
handles = []
handles.append(
    gscene.create_safe(gtype=GEOTYPE.BOX, name=box.name+"_hdl_tp_a", link_name="base_link", 
                   dims=(GRIP_DEPTH, box.dims[1],HANDLE_THICKNESS), center=(0,0,box.dims[2]/2-GRIP_DEPTH/2), rpy=(0,np.pi/2,0), 
                       color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                   parent=box.name)
)
action_points_dict = {"placement": PlacePoint("placement", box, [0,0,-box.dims[2]/2-CLEARANCE], [0,0,0])}
action_points_dict.update({handle.name: Grasp2Point(handle.name, handle, None, (0,0,0)) for handle in handles})
box_pobj = pscene.create_object(oname=box.name, gname=box.name, _type=CustomObject, action_points_dict=action_points_dict)

## Motion Planning

### moveit

In [15]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

In [16]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
ppline.set_motion(mplan)

In [17]:
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_sampler(tplan)

In [18]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [28]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

obj, handle, binder = "box", "box_hdl_tp_a", "grip0"
from_state = pscene.update_state([0]*6)
# from_state = pscene.update_state(crob.home_pose)
pscene.set_object_state(from_state)
to_state = pscene.make_goal_state(from_state, obj=obj, handle=handle, binder=binder)
print(from_state.binding_state)
print(to_state.binding_state)

(('box', 'placement', 'floor', 'floor'),)
(('box', 'box_hdl_tp_a', 'grip0', 'grip0'),)


In [29]:
# redundancy = sample_redundancy(
#     combine_redundancy(
#         pscene.subject_dict[obj].action_points_dict[handle], 
#         pscene.actor_dict[binder]))
redundancy={'box_hdl_tp_a': 
            {'w': np.pi/2,
             'x': 0.0,
             'y': 0.0},
            'grip0': 
            {'w': 0.0}}
Traj, end_state, error, success = ppline.test_connection(from_state, to_state, 
                                                    redundancy_dict={obj: redundancy}, timeout=5)
pscene.set_object_state(from_state)
print(success)

True


In [38]:
pscene.gscene.show_motion(Traj, period=0.05)

In [39]:
pscene.set_object_state(end_state)

In [32]:
state_home = end_state.copy(pscene)
state_home.Q = np.copy([0]*6)

In [33]:
trajectory, Q_last, error, success, binding_list = mplan.plan_transition(end_state, state_home, timeout=5)
print(success)

True


In [34]:
pscene.gscene.show_motion(trajectory, period=0.05)

In [41]:
pscene.gscene.show_motion(Traj, period=0.05)
pscene.set_object_state(from_state)
gscene.show_pose([0]*6)

In [42]:
pscene.set_object_state(from_state)
pscene.gscene.show_motion(Traj, period=0.05)
pscene.set_object_state(end_state)
pscene.gscene.show_motion(trajectory, period=0.05)

### eTaSL

In [21]:
from pkg.planning.motion.etasl.etasl import EtaslPlanner
eplan = EtaslPlanner(pscene)
eplan.update_gscene()

In [22]:
traj, lastQ, error, success, binding_list = eplan.plan_transition(from_state, to_state, 
                                                    N=1000)
print(success)

True


In [23]:
gscene.show_motion(traj, period=0.05)

## Disconnect stereo

In [20]:
stereo.disconnnect()